In [11]:
import pandas as pd
import os
import torch 
from transformers import CLIPTextModel, CLIPTextConfig,AutoProcessor, CLIPModel

import numpy as np
from PIL import Image
import string

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device,"device")

cuda device


In [12]:
summary_bert_path = "/users/eleves-b/2022/hanna.mergui/Computer-Vision/ComputerVision_Data/Summaries/export_summary_Bert.csv"
summary_bert = pd.read_csv(summary_bert_path)


summary_bert_ingredients_path="/users/eleves-b/2022/hanna.mergui/Computer-Vision/ComputerVision_Data/Summaries/export_summary_bert_with_ingredients.csv"
summary_bert_ingredients=pd.read_csv(summary_bert_ingredients_path)

data_dir = "ComputerVision_Data"
images_dir = "Food Images/Food Images"

In [13]:
recipe = summary_bert["Summary"]
print("number of sumary recipe ",len(recipe))

images = os.listdir(os.path.join(data_dir, images_dir))
print("number of images ",len(images))

number of sumary recipe  13463
number of images  3486


In [14]:
#max_position_embeddings=2048 is used to increase the maximum number of tokens that can be used in the model

# Initializing a CLIPTextModel with openai/clip-vit-base-patch32 style configuration
configuration = CLIPTextConfig(max_position_embeddings=2048)
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")


# Initializing a CLIPTextConfig from the openai/clip-vit-base-patch32 style configuration
model = CLIPTextModel(configuration)

# Accessing the model configuration
configuration = model.config

print(configuration)

CLIPTextConfig {
  "attention_dropout": 0.0,
  "bos_token_id": 49406,
  "eos_token_id": 49407,
  "hidden_act": "quick_gelu",
  "hidden_size": 512,
  "initializer_factor": 1.0,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "clip_text_model",
  "num_attention_heads": 8,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "projection_dim": 512,
  "transformers_version": "4.38.1",
  "vocab_size": 49408
}



In [15]:

#Tensor creation and array creation

import string

context_length=77
array_text=[]


# Creation of a tensor for the text 
device = "cuda" if torch.cuda.is_available() else "cpu"
#print(device,"device")
text_tensor = torch.zeros(len(summary_bert), context_length, dtype=torch.long)
labels = []
for i, row in enumerate(summary_bert.iterrows()):
    summary = row[1]["Summary"]

   
    #print(len(summary))
    #print(summary)

    to_supress = string.punctuation + "—" + "–" + "()"
    summary = summary.translate(str.maketrans("", "",to_supress)) # Remove punctuation from summary
    #print(summary)
            
    array_text.append(summary)  # Access row data using integer indices
    labels.append(row[1]["Image_Name"])  # Access row data using integer indices

#torch.save(text_tensor, "text_tensor.pt")
    
print(len(array_text))


13463
0


In [16]:
image = Image.open("ComputerVision_Data/Food Images/Food Images/-bloody-mary-tomato-toast-with-celery-and-horseradish-56389813.jpg")

inputs = processor(
    text=array_text, images=image, return_tensors="pt", padding=True
)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

print(probs)


Token indices sequence length is longer than the specified maximum sequence length for this model (102 > 77). Running this sequence through the model will result in indexing errors


TypeError: forward() got an unexpected keyword argument 'pixel_values'

13463
13406


In [19]:
text_tensor = torch.load("text_tensor.pt")
print(text_tensor.shape)
print(len(array_text))

torch.Size([13463, 77])
13463


In [20]:
images_inputs=[]

for im in images:
    im_path = os.path.join(data_dir, images_dir, im)
    images_inputs.append(preprocess(Image.open(im_path)))
#images_inputs_tensor = torch.tensor(np.stack(images_inputs)).to(device)

In [21]:
print("text_tensor shape", text_tensor.shape)
#print("images_inputs_tensor shape ", images_inputs_tensor.shape)

print("images array shape",len(images_inputs))

text_tensor shape torch.Size([13463, 77])
images array shape 3486


In [23]:
text = clip.tokenize(["a diagram", "a dog", "a cat"],context_length).to(device)
print(type(text))
print(text.shape)


<class 'torch.Tensor'>
torch.Size([3, 77])


In [24]:

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


image = preprocess(Image.open("ComputerVision_Data/Food Images/Food Images/-bloody-mary-tomato-toast-with-celery-and-horseradish-56389813.jpg")).unsqueeze(0).to(device)
#text = clip.tokenize(["a diagram", "a dog", "a cat"]).to(device)


print(type(array_text))#list
print(len(array_text)) #13463


text = clip.tokenize(array_text).to(device)

print("image",image.shape) #torch.Size([1, 3, 224, 224])
print("text",text.shape) #torch.Size([13463, 150])

with torch.no_grad():
   
    image_features = model.encode_image(image)
    print("type image_features",type(image_features.shape)) #<class 'torch.Size'>
    print("shape image_features",image_features.shape) #torch.Size([1, 1024]
    
    text_features = model.encode_text(text)
    print("type text_features",type(text_features.shape)) #<class 'torch.Size'>
    print("shape text_features",text_features.shape)
    
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()



#Note : normal qu'il y a un out of memory, je donne une liste et pas un tensor. Il faut changer ça. 


<class 'list'>
13463


image torch.Size([1, 3, 224, 224])
text torch.Size([13463, 77])
type image_features <class 'torch.Size'>
shape image_features torch.Size([1, 1024])


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.98 GiB. GPU 0 has a total capacity of 11.74 GiB of which 329.94 MiB is free. Including non-PyTorch memory, this process has 11.30 GiB memory in use. Of the allocated memory 8.41 GiB is allocated by PyTorch, and 2.76 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)